In [ ]:
"""
Dask 사용법 기초

원래 Pandas 는 데이터베이스나 CSV 파일의 데이터를 
모두 메모리로 읽어들인 다음 메모리 위에서 데이터를 처리한다. 
하지만 데이터의 양이 많은 경우에는 메모리의 제한으로 데이터프레임을 
만들 수 없는 경우가 있다. 
또한 데이터프레임의 크기가 너무 크면 질의나 그룹 연산을 할 때 
하나의 CPU 코어로 처리하기에는 시간이 너무 많이 걸릴 수도 있다.

이러한 경우에 도움이 되는 것이 Dask 패키지이다. 
Dask 패키지는 Pandas 데이터프레임 형식으로 
빅데이터를 처리하기 위한 파이썬 패키지로 다음과 같은 두 가지 기능을 가진다.
1. 가상 데이터프레임
2. 병렬처리용 작업 스케줄러
"""

In [1]:
"""
가상 데이터프레임

Dask 패키지를 사용하면 가상 데이터프레임을 만들 수 있다. 
가상 데이터프레임은 Pandas 데이터프레임과 비슷한 기능을 제공하지만 
실제로 모든 데이터가 메모리 상에 로드되어 있는 것이 아니라 
하나 이상의 파일 혹은 데이터베이스에 존재하는 채로 처리할 수 있는 기능이다. 

따라서 메모리 크기와 관계 없이 엄청나게 큰 CSV 파일을 
가상 데이터프레임으로 로드하거나 
같은 형식의 데이터를 가진 여러개의 CSV 파일을 
하나의 가상 데이터프레임에 로드할 수 있다.

실제로 Dask의 가상 데이터프레임을 
어떻게 쓸 수 있는지 실습을 통해 알아보자. 
실습을 위해 홈 디렉토리 아래에 data라는 임시 디렉토리를 만들고 
그곳으로 옮겨 CSV 파일을 하나 만들어 보자.
"""

In [2]:
# make folder dask
!mkdir dask

In [3]:
%cd dask

C:\pycharm\dask


In [5]:
%%writefile data1.csv
time,temperature,humidity
0,22,58
1,21,57
2,25,57
3,26,55
4,22,53
5,23,59

Writing data1.csv


In [6]:
# dask package의 dataframe import
import dask.dataframe as dd


In [7]:
# read_csv 명령으로 데이터 파일 data1.csv에 
# 대한 가상 데이터프레임 df를 만든다.
df = dd.read_csv("data1.csv")
df

,time,temperature,humidity
npartitions=1,,,
None,int64,int64,int64
None,...,...,...


In [8]:
# df는 데이터프레임과 유사하지만 실제로 데이터를 
# 메모리에 읽지 않았기 때문에 값은 표시되지 않는다.

# head, tail 명령을 내리면 그 때서야 일부 데이터를 읽어서 표시한다.
df.head()

,time,temperature,humidity
0,0,22,58
1,1,21,57
2,2,25,57
3,3,26,55
4,4,22,53


In [10]:
# temperature 열의 평균을 구해보자.
df.temperature.mean()

dd.Scalar<series-..., dtype=float64>

In [ ]:
"""
데이터프레임과 달리 바로 결과가 나오지 않는다. 
그 이유는 연산 반환값이 결과가 아닌 작업(task)이기 때문이다. 
구체적으로 어떤 작업인지를 보려면 visualize 메서드를 사용하여 
작업 그래프(graph)를 볼 수 있다.
 
작업 그래프란 이 계산을 하기 위해 
실제로 CPU가 해야 할 일들의 순서도라고 볼 수 있다.

이 작업의 값을 실제로 구하려면 
결과로 받은 작업 객체의 compute 메서드를 호출해야 한다.
"""

In [11]:
df.temperature.mean().compute()

23.166666666666668

In [12]:
# 이번에는 이 값을 화씨로 변환해 보자.
(df.temperature * 9 / 5 +32).compute()

0    71.6
1    69.8
2    77.0
3    78.8
4    71.6
5    73.4
Name: temperature, dtype: float64